# Deploying a model with live microphone input

In this example we will walk through loading a model exported from NengoEdge,
and deploying that model in a simple application that detects
keywords from live microphone input.

In order to run this example, the first step is to train a model in NengoEdge.
See [this blog post](TODO) for a detailed walkthrough on how to train such a model.
You can use whatever dataset you like, but make sure to match the parameters below
to your training data.

In [ ]:
import numpy as np
import sounddevice as sd

# Keyword labels for model outputs
labels = [
    "<silence>",
    "<unknown>",
    "yes",
    "no",
    "up",
    "down",
    "left",
    "right",
    "on",
    "off",
    "stop",
    "go",
]

# Sample rate of audio files in the dataset
sample_rate = 16000

After training completes successfully, go to the "Deployment" tab for that run, and
select the "SavedModel" or "TFLite" download option (the available export formats will
depend on which hardware device you selected for the run in NengoEdge). Extract the
downloaded archive, which contains
the data files for the model in the selected format. Set the path below to the directory
containing the extracted files.

In the example below we are running a model in SavedModel format, but if you downloaded
a model in TFLite format then simply switch the import below to `tflite_runner`.

In [ ]:
from nengo_edge.saved_model_runner import Runner

# from nengo_edge.tflite_runner import Runner

runner = Runner(".")

Next we need to select the microphone input device:

In [ ]:
print(f"Available devices:\n{sd.query_devices()}")
mic_device_ix = int(input("Index of input device to listen to: "))

And finally we can run the model using that device. As you speak into your microphone,
it will print out the keywords detected by the model.

Note that the particular input method below is not a requirement. The important part is
that we call `runner.run` in a loop, feeding it the new audio samples each time. This
can be adapted to whatever approach makes the most sense for your application.

In [ ]:
inference_rate = 0.2  # How often to do a classification, in seconds
block_size = int(sample_rate * inference_rate)


def callback(data, frames, time, status):
    # Call model periodically as new data comes in (taking advantage of internal
    # state for streaming)
    outputs = runner.run(data.T)
    label = labels[np.argmax(outputs)]
    if label != "<silence>":
        print(label)


try:
    with sd.InputStream(
        device=mic_device_ix,
        samplerate=sample_rate,
        blocksize=block_size,
        latency="low",
        channels=1,
        dtype=np.float32,
        callback=callback,
    ):
        print("Press Enter to quit")
        input()
    print("Exiting")
except KeyboardInterrupt:
    print("Exiting")